In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 4)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [5/200], loss:-1.6415
x:  tensor([[  6.8835, -34.7004, -13.1981,  41.1600]], grad_fn=<AddmmBackward>)
y: tensor([[  6.1929, -35.5965, -13.6622,  40.5128]], grad_fn=<AddmmBackward>)
b: tensor([[  7.8568, -36.6743, -12.0466,  40.7080]], grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.0524
x:  tensor([[ 45.1957, -29.4858,   3.1277,  28.8252]], grad_fn=<AddmmBackward>)
y: tensor([[ 45.8302, -29.8711,   3.1552,  28.9440]], grad_fn=<AddmmBackward>)
b: tensor([[ 48.5166, -30.5043,   4.7429,  27.8424]], grad_fn=<AddmmBackward>)
epoch [7/200], loss:-0.9917
x:  tensor([[ 46.3081, -42.3072, -10.3041,   5.2231]], grad_fn=<AddmmBackward>)
y: tensor([[ 45.8441, -42.5991, -10.4921,   4.8796]], grad_fn=<AddmmBackward>)
b: tensor([[ 50.5589, -42.4673,  -8.5554,   4.7300]], grad_fn=<AddmmBackward>)
epoch [8/200], loss:-1.3891
x:  tensor([[ 38.7877, -37.7639,  -2.7145,   8.4383]], grad_fn=<AddmmBackward>)
y: tensor([[ 39.1446, -36.0826,  -2.3097,   8.0067]], grad_fn=<AddmmBackward>)
b: tensor([[ 42

y: tensor([[26.1009,  5.9859, 26.2431, -4.8684]], grad_fn=<AddmmBackward>)
b: tensor([[29.7313,  8.8148, 26.5245, -7.0820]], grad_fn=<AddmmBackward>)
epoch [37/200], loss:-1.5358
x:  tensor([[36.1012, 17.8312, -0.9087, 29.0556]], grad_fn=<AddmmBackward>)
y: tensor([[36.0776, 19.1951, -0.5347, 29.0150]], grad_fn=<AddmmBackward>)
b: tensor([[39.0079, 19.9247, -1.0570, 26.3288]], grad_fn=<AddmmBackward>)
epoch [38/200], loss:-1.4161
x:  tensor([[ 16.5147,   2.2767, -23.0496,   3.1510]], grad_fn=<AddmmBackward>)
y: tensor([[ 15.7713,   1.0700, -22.8262,   2.8401]], grad_fn=<AddmmBackward>)
b: tensor([[ 17.5766,   3.7684, -22.6501,  -1.3698]], grad_fn=<AddmmBackward>)
epoch [39/200], loss:-1.3635
x:  tensor([[  7.8758,  16.0521, -14.6943, -14.2155]], grad_fn=<AddmmBackward>)
y: tensor([[  7.5486,  16.5241, -16.4498, -13.5635]], grad_fn=<AddmmBackward>)
b: tensor([[ 10.0751,  16.8566, -15.8307, -17.6183]], grad_fn=<AddmmBackward>)
epoch [40/200], loss:-1.4510
x:  tensor([[17.5071, 11.2745,  

epoch [68/200], loss:-1.4746
x:  tensor([[  4.0130, -22.8567,  20.7204, -46.0517]], grad_fn=<AddmmBackward>)
y: tensor([[  3.6996, -23.2017,  20.7449, -45.1602]], grad_fn=<AddmmBackward>)
b: tensor([[  4.8289, -22.5090,  17.1502, -46.0042]], grad_fn=<AddmmBackward>)
epoch [69/200], loss:-1.4169
x:  tensor([[-10.0393, -15.5303,  21.0278, -44.0686]], grad_fn=<AddmmBackward>)
y: tensor([[-10.3552, -15.8765,  20.9803, -43.2457]], grad_fn=<AddmmBackward>)
b: tensor([[ -9.4598, -14.5455,  18.1081, -44.6346]], grad_fn=<AddmmBackward>)
epoch [70/200], loss:-1.7191
x:  tensor([[  4.6936, -13.1865,  11.6456, -21.9660]], grad_fn=<AddmmBackward>)
y: tensor([[  4.4406, -13.5158,  11.6454, -21.0364]], grad_fn=<AddmmBackward>)
b: tensor([[  3.3022, -12.7854,   9.2726, -23.5865]], grad_fn=<AddmmBackward>)
epoch [71/200], loss:-1.6298
x:  tensor([[ -4.6415,   2.8969,  25.6804, -12.3331]], grad_fn=<AddmmBackward>)
y: tensor([[ -4.9264,   2.5116,  25.6862, -11.3567]], grad_fn=<AddmmBackward>)
b: tensor([

epoch [99/200], loss:-1.6285
x:  tensor([[ 15.2788, -22.3871,  -6.8257,  21.6799]], grad_fn=<AddmmBackward>)
y: tensor([[ 14.9931, -22.7175,  -6.8063,  22.5267]], grad_fn=<AddmmBackward>)
b: tensor([[ 12.7371, -19.9265,  -8.0449,  19.3668]], grad_fn=<AddmmBackward>)
epoch [100/200], loss:-1.4610
x:  tensor([[ 25.1513, -11.1699,   5.7434, -15.2868]], grad_fn=<AddmmBackward>)
y: tensor([[ 25.7926, -11.4040,   5.8323, -15.7047]], grad_fn=<AddmmBackward>)
b: tensor([[ 24.9200, -10.0827,   4.5962, -16.0654]], grad_fn=<AddmmBackward>)
epoch [101/200], loss:-1.4564
x:  tensor([[  9.6329, -23.0052,  31.2332, -10.2809]], grad_fn=<AddmmBackward>)
y: tensor([[  9.2967, -22.1409,  30.9331, -11.0870]], grad_fn=<AddmmBackward>)
b: tensor([[ 10.2992, -19.4248,  27.2849, -11.1786]], grad_fn=<AddmmBackward>)
epoch [102/200], loss:-1.5134
x:  tensor([[ 16.3443, -15.3173,  28.4559, -30.0842]], grad_fn=<AddmmBackward>)
y: tensor([[ 16.9711, -15.5164,  28.5415, -30.5246]], grad_fn=<AddmmBackward>)
b: tenso

b: tensor([[  8.6189, -16.2633,  11.0747,  20.4662]], grad_fn=<AddmmBackward>)
epoch [131/200], loss:-1.6775
x:  tensor([[-20.4237,   5.1948,  30.8935,  -7.4408]], grad_fn=<AddmmBackward>)
y: tensor([[-19.6317,   5.0223,  31.2526,  -8.3726]], grad_fn=<AddmmBackward>)
b: tensor([[-17.3453,   4.1359,  26.6043,  -6.8211]], grad_fn=<AddmmBackward>)
epoch [132/200], loss:-1.4713
x:  tensor([[  3.2459,  28.6854,  45.2748, -13.5915]], grad_fn=<AddmmBackward>)
y: tensor([[  3.5594,  28.7666,  43.3910, -13.0101]], grad_fn=<AddmmBackward>)
b: tensor([[  3.9763,  26.3409,  39.2944, -12.9896]], grad_fn=<AddmmBackward>)
epoch [133/200], loss:-1.5076
x:  tensor([[  9.7426,  16.0056,  27.9709, -16.7725]], grad_fn=<AddmmBackward>)
y: tensor([[ 10.0923,  16.1010,  26.0333, -16.2068]], grad_fn=<AddmmBackward>)
b: tensor([[ 10.1330,  15.7567,  23.3285, -17.2994]], grad_fn=<AddmmBackward>)
epoch [134/200], loss:-1.4887
x:  tensor([[28.7577,  7.5047, 37.3618, -1.1053]], grad_fn=<AddmmBackward>)
y: tensor([

y: tensor([[ 19.6461, -56.0724,  47.8966, -11.3613]], grad_fn=<AddmmBackward>)
b: tensor([[ 16.8809, -53.8033,  42.7531, -11.1298]], grad_fn=<AddmmBackward>)
epoch [162/200], loss:-1.3678
x:  tensor([[ 33.2543, -67.5146,  44.0604, -12.7490]], grad_fn=<AddmmBackward>)
y: tensor([[ 33.9777, -67.6617,  44.4017, -13.6961]], grad_fn=<AddmmBackward>)
b: tensor([[ 31.2056, -65.0504,  40.4008, -13.3885]], grad_fn=<AddmmBackward>)
epoch [163/200], loss:-1.5005
x:  tensor([[ 22.6721, -33.3217,  57.9960,  10.2817]], grad_fn=<AddmmBackward>)
y: tensor([[ 21.9018, -34.5175,  58.1493,  10.1053]], grad_fn=<AddmmBackward>)
b: tensor([[ 20.2330, -32.3603,  51.9637,   7.3044]], grad_fn=<AddmmBackward>)
epoch [164/200], loss:-1.5068
x:  tensor([[ 18.7537, -31.8537,  43.0237,   4.9631]], grad_fn=<AddmmBackward>)
y: tensor([[ 19.3510, -32.0132,  43.1845,   4.4655]], grad_fn=<AddmmBackward>)
b: tensor([[ 16.4559, -30.3740,  38.6073,   3.9654]], grad_fn=<AddmmBackward>)
epoch [165/200], loss:-1.5108
x:  tens

b: tensor([[ 22.5267, -21.7261,  28.9015,  -4.4190]], grad_fn=<AddmmBackward>)
epoch [193/200], loss:-1.5131
x:  tensor([[13.4659,  4.1202, 36.5863,  2.6388]], grad_fn=<AddmmBackward>)
y: tensor([[13.0587,  5.0462, 36.2186,  1.7925]], grad_fn=<AddmmBackward>)
b: tensor([[13.7913,  5.4591, 31.7981,  2.5126]], grad_fn=<AddmmBackward>)
epoch [194/200], loss:-1.5536
x:  tensor([[ 9.6038, -4.1969, 19.3452, 15.4877]], grad_fn=<AddmmBackward>)
y: tensor([[ 9.6508, -3.9507, 20.1566, 16.2489]], grad_fn=<AddmmBackward>)
b: tensor([[ 8.3030, -3.9104, 16.2013, 14.3887]], grad_fn=<AddmmBackward>)
epoch [195/200], loss:-1.4339
x:  tensor([[-14.9228,  13.9435,  36.8336,  12.7265]], grad_fn=<AddmmBackward>)
y: tensor([[-15.0729,  13.4970,  36.7442,  13.5997]], grad_fn=<AddmmBackward>)
b: tensor([[-14.5329,  12.5860,  31.3363,  11.4154]], grad_fn=<AddmmBackward>)
epoch [196/200], loss:-1.3632
x:  tensor([[ 5.8602,  7.7269, 32.0801, 12.9332]], grad_fn=<AddmmBackward>)
y: tensor([[ 5.8847,  7.8702, 32.81

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [2]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time
print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(3):
    z = cv2.imread('Images1/2f.png')
    z  = cv2.resize(z,(100,100))
    img = z#uframes[i]
    print(np.shape(img))
    ms = Image.fromarray(img)
    ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


(100, 100, 3)
Length of Class data:  600
Start
(100, 100, 3)
"Drone Stationary, Tank Firing"
(100, 100, 3)
"Drone Stationary, Tank Firing"
(100, 100, 3)
"Drone Stationary, Tank Firing"


In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4